In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline
import  matplotlib.pyplot as plt
import joblib
from sklearn.metrics import mean_absolute_percentage_error
import random
import torch
from torch import  nn,optim
import os
from torch.utils import data
import math
from sklearn.preprocessing import MinMaxScaler

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=521)


seq = 32
hidden_size = 32
num_layers = 8
batch_size = 32
epochs = 100

train= pd.read_excel('./dataset/train.xlsx')
test= pd.read_excel('./dataset/test.xlsx')
Y_columns =['TNeff']
X_columns = [col for col in train.columns if col not in ['time','TNeff']]

def load_data():
    train = pd.read_excel('./dataset/test.xlsx')
    test = pd.read_excel('./dataset/test.xlsx')


    time_columns=['time']
    X_columns = [col for col in train.columns if col not in ['time','TNeff']]
    Y_columns =['TNeff']

    x_train = train[X_columns]
    y_train = train[Y_columns]
    x_test = test[X_columns]
    y_test = test[Y_columns]
    return x_train, y_train, x_test, y_test, X_columns, Y_columns

# class EI:
#     def __init__(self,trainin,testin):
#         train=self.trainin
#         test=self.testin
#         adi=self.ad(adr=0.2,trains=train,threshold=0.03)
#         trainad=train.drop(index=adi).reset_index(drop=True)
#         train,test=self.ts(trainad,test)
#     def ad(adr,trains,sdel):
#         ei=[]
#         train=trains
#         index=[i for i in train.columns][1:-1]
#         threshold_num=sdel*len(trains)
#         for ind in index:
#             top,low=train.loc[:, ind].max(),train.loc[:, ind].min()
#             diff=top-low
#             diffmin,diffmax=low+adr*diff,low+(1-adr)*diff
#             minlist,maxlist=train[train[inx] < diffmin].index.values.tolist(),train[train[inx] > diffmax].index.values.tolist()
#             if len(minlist) <= threshold_num:
#                 for minindex in train[train[inx] < diffmin].index.values.tolist():
#                     ei.append(minindex)
#             if len(maxlist) <= threshold_num:
#                 for maxindex in train[train[inx] > diffmax].index.values.tolist():
#                     ei.append(maxindex)
#         ei=list(set(ei))
#         ei=sorted(ei)
#         return ei
#     def ts(trains,tests):
#         train=trains
#         test=tests
#         data = pd.concat([train, test], axis=0)
#
#         roll_cols = train.columns
#         for i in [j for j in range(1, 6)] + [12, 24, 48, 60]:
#             data[[ii+f'_mean_{i}' for ii in roll_cols]] = data[roll_cols].rolling(i, min_periods=1).mean()
#
#         pd.set_option("display.max_columns", 100)
#         save_rate = 0.5
#         data = data[data.describe().T.sort_values(by="std", ascending=False, inplace=False).iloc[: int(len(data.columns)*save_rate)].index.tolist()]
#
#         traint = data.iloc[:train.shape[0]].reset_index(drop=True)
#         testt = data.iloc[train.shape[0]:].reset_index(drop=True)
#         return traint,testt
#     def adts(self):
#         return train,test
# ei=EI(train,test)
# train,test=EI.adts()

x_train=train[X_columns]
x_test=test[X_columns]
y_train=train[Y_columns]
y_test=test[Y_columns]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("*********************")
print(torch.cuda.get_device_name(0))
print(device)

def split_data(train_sca, test_sca, seq, X_columns, Y_columns):  # 47预测1
    data_raw = pd.concat([train_sca, test_sca])

    data = []
    for index in range(len(data_raw) - seq):
        data.append(data_raw[index:index + seq])
    data = np.array(data)
    # print(len(data))  #14640
    test_set_size = len(test_sca)  # 96
    train_set_size = data.shape[0] - test_set_size

    # 第一个-1是47预测1
    # 第二个1是不计time，-1是只预测AQ
    x_train = data[:train_set_size, :-1, :len(X_columns)].astype('float32')
    y_train = data[:train_set_size, -1, -len(Y_columns)].reshape(-1, len(Y_columns)).astype('float32')

    x_test = data[train_set_size:, :-1, :len(X_columns)].astype('float32')
    y_test = data[train_set_size:, -1, -len(Y_columns)].reshape(-1, len(Y_columns)).astype('float32')

    return torch.tensor(x_train), torch.tensor(y_train), torch.tensor(x_test), torch.tensor(y_test)


def preprocess(x_train, y_train, x_test, y_test, X_columns, Y_columns):
    scaler1 = MinMaxScaler(feature_range=(0.000000001, 1))
    scaler2 = MinMaxScaler(feature_range=(0.000000001, 1))
    scaler3 = MinMaxScaler(feature_range=(0.000000001, 1))
    scaler4 = MinMaxScaler(feature_range=(0.000000001, 1))
    train_sca = pd.DataFrame()
    test_sca = pd.DataFrame()
    train_sca[X_columns] = scaler1.fit_transform(x_train)
    train_sca[Y_columns] = scaler2.fit_transform(y_train)
    test_sca[X_columns] = scaler3.fit_transform(x_test)
    test_sca[Y_columns] = scaler4.fit_transform(y_test)
    return train_sca, test_sca, (scaler1,scaler2, scaler3, scaler4)

x_train, y_train, x_test, y_test, X_columns, Y_columns = load_data()
train_sca, test_sca, (scaler1, scaler2, scaler3, scaler4) = preprocess(x_train, y_train, x_test, y_test, X_columns, Y_columns)
x_train_ts,y_train_ts,x_test_ts,y_test_ts = split_data(train_sca, test_sca, seq, X_columns, Y_columns)
input_size, output_size = len(X_columns), len(Y_columns)



input_size=len(X_columns)
output_size=len(Y_columns)




class FeedForward(nn.Module):
    def __init__(self, model_dim=512, hidden_dim=2048, dropout_rate=0.0):
        super().__init__()
        self.model_dim = model_dim
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate

        self.linear1 = nn.Linear(self.model_dim, self.hidden_dim)
        self.linear2 = nn.Linear(self.hidden_dim, self.model_dim)
        self.norm = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        output = self.linear2(torch.relu(self.linear1(x)))
        output = self.dropout(output)
        output = self.norm(output + x)
        return output


class GRUAttn(nn.Module):
    def __init__(self, input_dim, model_dim, ff_hidden_dim, num_heads=1, num_blocks=1,
                 dropout_rate=0.1):
        super(GRUAttn, self).__init__()
        self.num_blocks = num_blocks

        gru_hidden_dim = model_dim // 2
        self.gru = nn.GRU(input_dim, gru_hidden_dim, batch_first=True, bidirectional=True)

        for i in range(self.num_blocks):
            self.__setattr__('multihead_attn_{}'.format(i), nn.MultiheadAttention(model_dim, num_heads))
            self.__setattr__('feedforward_{}'.format(i), FeedForward(model_dim=model_dim,
                                                                     hidden_dim=ff_hidden_dim,
                                                                     dropout_rate=dropout_rate))

        self.fc = nn.Linear(model_dim, 1)

    def forward(self, x):
        x, _ = self.gru(x)  # x (B, T, 2 * D/2)
        for i in range(self.num_blocks):
            x, _ = self.__getattr__('multihead_attn_{}'.format(i))(x, x, x)
            x = self.__getattr__('feedforward_{}'.format(i))(x)

        x = self.fc(x)
        return x[:, -1, :]
def get_model_output(model, input, bs=128):
    with torch.no_grad():
        for i in range(math.ceil(input.shape[0]/bs)):
            Y_pred = model(input[i*bs:(i+1)*bs]).detach().cpu().numpy().reshape(-1, 1)
            if i == 0:
                res = Y_pred
            else:
                res = np.concatenate([res, Y_pred], 0)
    return res
def train(model, model_path, device, train_iter, x_test, y_test, scaler4):
    myloss = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    import time
    start_time = time.time()

    lossmin = 1
    # for epoch in range(epochs):
    #     for inx, (X, Y) in enumerate(train_iter):
    #         Y_pred = model(X)
    #         loss = myloss(Y, Y_pred)
    #         # 梯度清0
    #         optimizer.zero_grad()
    #         # 计算梯度
    #         loss.backward()
    #         # 修改权值
    #         optimizer.step()
    #         torch.cuda.empty_cache()
    #
    #     y_test_pred = get_model_output(model, x_test.to(device))
    #     y_test_pred = scaler4.inverse_transform(y_test_pred)
    #     loss = mean_absolute_percentage_error(y_test, y_test_pred)
    #     print(f'*****************epoch{epoch} MAPE {loss}')
    #     if loss < 1 and loss < lossmin:
    #         lossmin = loss
    #         torch.save(model.state_dict(), model_path)

    training_time = time.time() - start_time
    print(f'training_time{training_time}')
    return


torch_dataset = data.TensorDataset(x_train_ts.to(device), y_train_ts.to(device))
train_iter = data.DataLoader(
    dataset=torch_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)


model_path = './mpkl/m15_at_.pkl'
model = GRUAttn(input_size, hidden_size, 32).to(device)


# train(model, model_path, device, train_iter, x_test_ts, y_test, scaler4)
model.load_state_dict(torch.load(model_path))



y_train_pred = get_model_output(model, x_train_ts.to(device))
y_train_pred = scaler2.inverse_transform(y_train_pred)
y_train_pred = pd.DataFrame(columns=['AQ'], data=y_train_pred)

y_test_pred = get_model_output(model, x_test_ts.to(device))
y_test_pred = scaler4.inverse_transform(y_test_pred)
y_test_pred = pd.DataFrame(columns=['AQ'], data=y_test_pred)
print(f'***************************************************')
loss = mean_absolute_percentage_error(y_test, y_test_pred)
print('MAPE:', loss)









x_train=pd.DataFrame(y_train_pred)
x_test=pd.DataFrame(y_test_pred)
y_train=y_train[seq:]
y_test=y_test[seq:]


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x_train = torch.from_numpy(np.float32(x_train)).to(device)
y_train = torch.from_numpy(np.float32(y_train)).to(device)
x_test = torch.from_numpy(np.float32(x_test)).to(device)
y_test = torch.from_numpy(np.float32(y_test)).to(device)

class ATN(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim):
        super(ATN, self).__init__()

		#定义层
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.sig1 = nn.Sigmoid() #nn.Linear为线性关系，加上激活函数转为非线性
        self.drop1=nn.Dropout(p=0.7)


        self.fc2 = nn.Linear(hidden_dim, output_dim)


    def forward(self, x):
        out = self.fc1(x)
        out = self.sig1(out)
        out = self.drop1(out)
        out = self.fc2(out)
        return out

input_dim = x_train.shape[1]  #一维向量长度
hidden_dim = 16            #hidden layer 神经元个数
output_dim = y_train.shape[1]  #1个类
batch_size=32
modelts = ATN(input_dim, hidden_dim, output_dim).to(device)
myloss = nn.MSELoss()     #均方差
optimizer = optim.Adam(modelts.parameters(),lr=0.001)
epochs =100
from torch.utils import data


torch_dataset = data.TensorDataset(x_train, y_train)
train_iter=data.DataLoader(
    dataset=torch_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

lossmin=1
import time
start_time=time.time()
for epoch in range(epochs):
    for inx,(X, Y) in enumerate(train_iter):
        Y_pred=modelts(X)
        loss =myloss(Y,Y_pred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    test_index=test.index
    y_test_pred_ts = modelts(x_test)
    y_test_pred = y_test_pred_ts.detach().cpu().numpy().reshape(-1,1)
    loss=mean_absolute_percentage_error(test[Y_columns],y_test_pred)
    print(f'*****************epoch{epoch} MAPE {loss}')

    if loss<1 and loss<lossmin:
        lossmin=loss
        torch.save(modelts.state_dict(),"./eiatnmpkl/m15_at_.pkl")

end_time=time.time()
t=end_time-start_time
y_test_pred_ts = modelts(x_test)
y_test_pred = y_test_pred_ts.detach().cpu().numpy().reshape(-1,1)
loss=mean_absolute_percentage_error(test[Y_columns],y_test_pred)


In [ ]:
frame=pd.DataFrame()
frame['ytrain']=pd.DataFrame(y_train_pred)
frame['ytest']=pd.DataFrame(y_test_pred)
frame.to_excel('./modelresult/m15_at.xlsx')